In [37]:
#from ipycytoscape import CytoscapeWidget
#import ipywidgets
#import networkx as nx
#import panel as pn

#pn.extension('ipywidgets')

#G = nx.complete_graph(5)
#cyto = CytoscapeWidget()
#cyto.graph.add_graph_from_networkx(G)
#wi = pn.pane.IPyWidget(cyto).servable()
#wi

In [12]:
#import panel as pn
#pn.extension()



#from dash import Dash, html
#import dash_cytoscape as cyto

# Create the Dash app
#app = Dash(__name__)

# Define the layout using Dash components
#layout = html.Div([
#    cyto.Cytoscape(
#        id='cytoscape-elements-basic',
#        layout={'name': 'preset'},
#        style={'width': '100%', 'height': '400px'},
#        elements=[
#            # The nodes elements
#            {'data': {'id': 'one', 'label': 'Node 1'},
#             'position': {'x': 50, 'y': 50}},
#            {'data': {'id': 'two', 'label': 'Node 2'},
#             'position': {'x': 200, 'y': 200}},
#            {'data': {'id': 'three', 'label': 'Node 3'},
#             'position': {'x': 350, 'y': 350}},

            # The edge elements
#            {'data': {'source': 'one', 'target': 'two', 'label': 'Node 1 to 2'}}
#        ]
#    )
#])

# Set the layout of the Dash app
#app.layout = layout

#app.run()


#tree_view = pn.pane.HTML("""
#<iframe src="http://127.0.0.1:8050/" id="myFrame" height="600" width="900" allow_embedding=True>
#</iframe>""",
#styles={'background-color': '#F6F6F6',
#'border': '2px solid black',
#'border-radius': '5px',
#'padding': '10px'})  
#tree_view.show()

Launching server at http://localhost:46809


In [2]:
#import dash_bio as dashbio
#from dash import Dash, html, Input, Output, callback
#import urllib.request as urlreq
#import os
#import panel as pn



#file_folder= "/home/infosebi/Documents/Programmieren/MastersProject/test_5/aligned/Tsac_0432.fasta"
#data = open(file_folder, "r").read()
#print(data) 
#app = Dash(__name__)
 

#data = urlreq.urlopen(
#    'https://git.io/alignment_viewer_p53.fasta'
#).read().decode('utf-8')
#print(data)

#app.layout = html.Div([
#    dashbio.AlignmentChart(
#        id='my-default-alignment-viewer',
#        data=data,
#        height=900,
#        tilewidth=30,
#    ),
#    html.Div(id='default-alignment-viewer-output')
#])

#@callback(
#    Output('default-alignment-viewer-output', 'children', allow_duplicate=True),
#    Input('my-default-alignment-viewer', 'eventDatum'), prevent_initial_call='initial_duplicate'
#)
#def update_output(value):
#    if value is None:
#        return 'No data.'

#button = pn.widgets.Button(name='Click me', button_type='primary')

#app.run(debug=True)
    

#alignment_viewer = pn.pane.HTML("""
#<iframe src="http://127.0.0.1:8050/" id="myFrame" height="900" width="1200" allow_embedding=True>
#</iframe>""",
#styles={'background-color': '#F6F6F6',
#'border': '2px solid black',
#'border-radius': '5px',
#'padding': '10px'}, height=900, width=1200)

#pn.Row(alignment_viewer, button).show()


Launching server at http://localhost:39655


In [18]:
import os, io, random
import string
import numpy as np

from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

import panel as pn
import panel.widgets as pnw
pn.extension()

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot

def get_colors(seqs):
    """make colors for bases in sequence"""
    text = [i for s in list(seqs) for i in s]
    clrs =	{'A': 'lightgreen', 'G': 'lightgreen', 'C': 'green', 'D': 'darkgreen', 'E': 'darkgreen', 'N': 'darkgreen', 'Q': 'darkgreen',
            'I': 'blue', 'L': 'blue', 'M': 'blue', 'V': 'blue', 'F': 'palevioletred', 'W': ' palevioletred', 'Y': ' palevioletred', 'H': 'darkblue', 'K': 'orange',
            'R': 'orange', 'P': 'pink', 'S': 'red', 'T': 'red', '-': 'white'}
    colors = [clrs[i] for i in text]
    return colors


def view_alignment(aln, fontsize="9pt", plot_width=800):
    """Bokeh sequence alignment view"""

    #make sequence and id lists from the aln object
    seqs = [rec.seq for rec in (aln)]
    ids = [rec.id for rec in aln]    
    text = [i for s in list(seqs) for i in s]
    colors = get_colors(seqs)    
    N = len(seqs[0])
    S = len(seqs)    
    width = .4

    x = np.arange(1,N+1)
    y = np.arange(0,S,1)
    #creates a 2D grid of coords from the 1D arrays
    xx, yy = np.meshgrid(x, y)
    #flattens the arrays
    gx = xx.ravel()
    gy = yy.flatten()
    #use recty for rect coords with an offset
    recty = gy+.5
    h= 1/S
    #now we can create the ColumnDataSource with all the arrays
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors ))#colors=colors
    plot_height = len(seqs)*15+50
    x_range = Range1d(0,N+1, bounds='auto')
    if N>100:
        viewlen=100
    else:
        viewlen=N
    #view_range is for the close up view
    view_range = (0,viewlen)
    tools="xpan, xwheel_zoom, reset, save"

    #entire sequence view (no text, with zoom)
    p = figure(title=None,  height=50, width=plot_width,   #plot_width= plot_width,
               x_range=x_range, y_range=(0,S), tools=tools,
               min_border=0, toolbar_location='below')
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                 line_color=None, fill_alpha=0.6)
    p.add_glyph(source, rects)
    p.yaxis.visible = False
    p.grid.visible = False  

    #sequence text view with ability to scroll along x axis
    p1 = figure(title=None, width=plot_width, height=plot_height,
                x_range=view_range, y_range=ids, tools="xpan,reset",
                min_border=0, toolbar_location='below')#, lod_factor=1)          
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black", text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                line_color=None, fill_alpha=0.4)
    p1.add_glyph(source, glyph)
    p1.add_glyph(source, rects)

    p1.grid.visible = False
    p1.xaxis.major_label_text_font_style = "bold"
    p1.yaxis.minor_tick_line_width = 0
    p1.yaxis.major_tick_line_width = 0

    p = gridplot([[p],[p1]], toolbar_location='below')
    return p

aln = AlignIO.read("/home/infosebi/Documents/Programmieren/MastersProject/test_5/aligned/Tsac_0432.fasta",'fasta')
p = view_alignment(aln, plot_width=900)
pn.pane.Bokeh(p).servable()



Bokeh(GridPlot)